In [67]:
using Pkg;
Pkg.add(url="https://github.com/FerreolS/OptimPackNextGen.jl",rev="verbose")

   Updating git-repo `https://github.com/FerreolS/OptimPackNextGen.jl`


   Updating registry at `~/.julia/registries/General`
######################################################################### 100,0%###############   99,6%
  Resolving package versions...
Updating `~/.julia/environments/v1.5/Project.toml`
  [080ffe24] ~ OptimPackNextGen v0.1.0 `https://github.com/FerreolS/OptimPackNextGen.jl#verbose` ⇒ v0.1.0 `https://github.com/FerreolS/OptimPackNextGen.jl#verbose`
Updating `~/.julia/environments/v1.5/Manifest.toml`
  [080ffe24] ~ OptimPackNextGen v0.1.0 `https://github.com/FerreolS/OptimPackNextGen.jl#verbose` ⇒ v0.1.0 `https://github.com/FerreolS/OptimPackNextGen.jl#verbose`


In [1]:
include("../src/SphereIFSCalib.jl")

using TwoDimensional, Zygote, StatsBase, Plots, OptimPackNextGen, FITSIO, DelimitedFiles, Tables, DataFrames
using HypothesisTests #contient ChisqTest

plotly()

Plots.PlotlyBackend()

### Constantes

In [2]:
# wavelengths
const λ1 = 987.72e-9# laser 1 
const λ2 = 1123.71e-9# laser 2 
const λ3 = 1309.37e-9# laser 3
#const λ4 = 1545.10e-9  # laser 4  
λlaser = [λ1,λ2,λ3]
λ0 = mean(λlaser);# reference
wavelengthrange = LinRange(850e-9,1600e-9,50); # coarse wavelength range of the instrument

## Fit global sur l'ensemble des lenslets:

On definit les parametres qui servent a creer le modele  

In [3]:
#Bbox of the lenslet: box autour d'un set de 3 gaussiennes
bbox = BoundingBox(xmin=1012, ymin=1048, xmax=1018, ymax=1092);
a0 = [300.0 , 400.0, 500.0];
fwhm0 = [2.0, 2.0 , 2.0];

#affichage d'un exemple de trois gaussiennes avec la bbox sur les donnees reelles:
f3 = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_wave_corrected.fits")
data= read(f3[1]);
ldata = data[1012:1018 , 1048:1092]
heatmap(ldata, title = "Lenslet correspondant a la bbox initiale (exemple)")

#il manque les coeff C0 pour pouvoir fiter les gaussiennes, qu'on recupere dans des fichiers txt

<!DOCTYPE html>
 
 
 Plots.jl

### 1 lecture des fichiers txt

On veut recuperer en sortie de fonction la premiere colonne (c0, coef a l'ordre 0, soit le centre de la lenslet).

x(λ) = cx0 + cx1*((λ-λ0)/λ0) + cx2*((λ-λ0)/λ0)^2

y(λ) = cy0 + cy1*((λ-λ0)/λ0) + cy2*((λ-λ0)/λ0)^2

In [4]:
coeffx = readdlm("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/coef_pol_x.txt", header = false)
cx0 = coeffx[:,1];
cx1 = coeffx[:,2];
cx2 = coeffx[:,3];

coeffy = readdlm("/home/user/stage/HR_4796-HD_95086/Calibration_wave_spec/coef_pol_y.txt", header = false)
cy0 = coeffy[:,1];
cy1 = coeffy[:,2];
cy2 = coeffy[:,3];

In [74]:
histogram(cy1)

<!DOCTYPE html>
 
 
 Plots.jl

### 2 ouvrir et trouver positions dans l'image brute

In [5]:
#1/ IFS_sim_wave.fits: on ouvre le modele, qui a peu de BP, donc les fit des lenslets seront bons. f5 = FITS("/home/user/Pictures/IFS_sim_wave.fits");
#2/ IFS_calib_wave_corrected: "vrai" fichier de depart (a comparer avec 1/ eventuellement)
f5 = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_wave_corrected.fits", "r"); #donnees laser

In [6]:
#Read the data or a subset thereof from disk, and save it in a pre-allocated output array A.
A =zeros(2048,2048); #pre-allocate matrix A  Array{Float64}(undef,2048,2048)
read!(f5[1], A);

In [7]:
close(f5);

In [46]:
#Constantes
nb_fit = 5 #test sur quelques lenslets
ainit = [300. , 400. , 500.];
fwhminit = [2.0, 2.0 , 2.0];
largeur = 8;
hauteur = 44;
mcx1 = mean(cx1);
mcy1 = mean(cy1);
mcx2 = mean(cx2);
mcy2 = mean(cy2);
N = length(cx0) #Nombre de lenslet (18908)

laser =  LaserModel(λlaser,ainit,fwhminit);

In [47]:
#Initialiser tableaux de resultats
atab = zeros(3,N);
fwhmtab = zeros(3,N);
ctab = zeros(3,N);
lmod = Array{LensletModel}(undef,N);

In [69]:
#On parcours les lenslets
for i=1:nb_fit #i: numero de la bbox
   #Initialisation
    #xmin = cx0[i,1]-largeur lenslet, ymin = cy0[i,1]-hauteur lenslet
    #xmax = cx0[i,1]+largeur lenslet, ymax = cy0[i,1]+hauteur lenslet
    #Remarque: centre de l'image 0 = 1025x1025 pixel, donc il faut ajouter 1025 a xmin et xmax  
    xmin=(cx0[i,1]-largeur/2)+1025;
    ymin=cy0[i,1]-hauteur/2;
    xmax=(cx0[i,1]+largeur/2)+1025;
    ymax=cy0[i,1]+hauteur/2;
    if xmin<0 
        continue
    end
    if xmax>2048 
        continue
    end
    bbox = round(Int, BoundingBox(xmin, ymin, xmax, ymax));  
    lmod = LensletModel(λ0,laser.nλ-1, bbox);   
    Cinit = zeros(2,laser.nλ);
    Cinit[1:2,1:3] = [ [cy0[i,1] mcy1 mcy2]; [cx0[i,1] mcx1 mcx2] ]; #C0[1:2,1:3] =[ [cy0[i,1] cy1[i,2] cy2[i,3]]; [cx0[i,1] cx1[i,2] cx2[i,3]] ]     
    
   #Fit du polynome pour chaque lenslet: optimisation
    xinit = vcat([ainit[:],fwhminit[:],Cinit[:]]...);
    ldata = view(A, lmod.bbox);
    
    lkl = LikelihoodIFS(lmod,laser, ldata);#rajouter la precision avec la carte des BP: view(precision,lenslet.bbox)
    cost(x::Vector{Float64}) = lkl(x)
    xopt = vmlmb(cost, xinit; verb=50);
    
   #Sauvegarde des valeurs des parametres du modele dans des vecteurs
    #ce qu'on veut garder: lmod sous forme de vecteur, 1 modele de lenslet par lenslet
    (aopt,fwhmopt,copt) = (xopt[1:(laser.nλ)],xopt[(laser.nλ+1):(2*laser.nλ)],reshape(xopt[(2*laser.nλ+1):(4*laser.nλ)],2,:));
    ###########a finir: ajouter Nan ou missing quand mauvais fit dans les tableaux
    atab[:,i] = aopt
    fwhmtab[:,i] = fwhm
    ctab[:,i] = copt
    lmod[:,i] = lmod
    
end 

LoadError: [91mArgumentError: reducing over an empty collection is not allowed[39m

In [37]:
#Test avec donnees lamp pour voir si spectre decoupee ou pas sur quelques lenslet
f6 = FITS("/home/user/stage/HR_4796-HD_95086/IFS_calib_spec_corrected.fits", "r"); #donnees lamp
B =zeros(2048,2048); #pre-allocate matrix A  Array{Float64}(undef,2048,2048)
read!(f6[1], B);
bbox = BoundingBox(xmin=1012, ymin=1048, xmax=1018, ymax=1092);
ldata = view(B,bbox) # ex lenslet au centre de l'image [1012:1018, 1048:1092]
heatmap(ldata)

<!DOCTYPE html>
 
 
 Plots.jl

On voit que le spectre n'est pas decoupe.

Warning: ajouter un flag de qualité (bon/mauvais ajustements)

Methode 1: tracer des histogrammes de tous les coeff, visualisation des outliers. Ajustement d'une gaussienne, calcul de la moyenne de la variance, ce qui est superieur a 3σ est sorti.

Methode 2: test statistique par rapport au guess si methode 1 marche pas. Histogramme des valeurs de χ2

In [ ]:
h1 = histogram(atab);
h2 = histogram(fwhmtab);
h3 = histogram(ctab);

In [ ]:
#Moyenne variance
@show std(h1);
@show std(h2);
@show std(h3);

Selectionner les lenslets aberrantes

### 3 affichage

Affichage de quelques lenslet

In [ ]:
#loi de dispersion
wxy0 = lmod.dmodel.(wavelengthrange);
(wx0,wy0) = collect.( zip(wxy0...));

In [ ]:
lmod.dmodel.(λlaser) 
#heatmap(ldata, title = "Fit laser") #1/ pour IFS_sim_wave.fits
heatmap(clamp.(ldata,0,1500), title = "Fit laser")
plot!(wy0,wx0; label="dispersion law")

In [ ]:
@show(ainit,aopt);

@show(fwhminit,fwhmopt);

@show(cinit,copt);

In [ ]:
#Point de depart de l'optimiseur
UpdateDispModel(lmod.dmodel, cinit);
UpdateLaserModel(laser,ainit,fwhminit);
limageinit = LensletLaserImage(lmod,laser)

#heatmap(limageinit, title = "Initialisation")

In [ ]:
#Point d'arrivee de l'optimiseur
UpdateDispModel(lmod.dmodel, copt);
UpdateLaserModel(laser,aopt,fwhmopt);
limageopt = LensletLaserImage(lmod,laser)

#heatmap(limageopt, title = "Point d'arrivee de l'optimiseur")


In [ ]:
#Residus
#heatmap( ldata .- limageopt, title = "Residus")